In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Intersections
todo

## Intersector Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D92 Intersector Large.jpg" style="width: 600px; display: inline;">

In [ ]:
"""
Intersector Initialization
Two primary members are defined: _geom that stores the geometric results of an 
intersection, and log, a String that gives feedback to the calling context. The 
geometric results are also made available by the property results.
"""
class Intersector(object):
    def __init__(self):
        self._geom = []
        self.log = None
        self.tol = EPSILON
        
    @property
    def results(self):
        return self._geom

In [ ]:
"""
Intersect Method
Given two pieces of Decod.es geometry, a and b, this method attempts to 
intersect them by calling upon the appropriate pairwise intersection method, 
and returns a success boolean.
"""
def of(self,a,b,**kargs):
    # clear re-initializes the _geom collection
     self.clear()
    
    # white-list of types supported by intersection methods
    supported_types = [Plane,Circle,PGon,RGon,Line,Ray,Segment,PLine,Arc]
    # ensures the ordering of a and b by type
    a,b = sorted( [a,b], key = lambda obj: supported_types.index(type(obj)) )
    
    if type(a) == Plane:
        if type(b) == Line : return self._line_plane(b,a,**kargs)
        if type(b) == Ray : return self._ray_plane(b,a,**kargs)
        if type(b) == Segment : return self._seg_plane(b,a)
        if type(b) == PLine : return self._pline_plane(b,a)
        if type(b) == Circle : return self._circle_plane(b,a)
        if type(b) == Arc : return self._arc_plane(b,a)
        if type(b) == Plane : return self._plane_plane(b,a)
        
    if type(a) == Circle:
        if type(b) == Circle : return self._circle_circle(b,a)
        if isinstance(b, LinearEntity) : return self._line_circle(b,a)
    
    if type(a) == RGon or type(a) == PGon:
        if isinstance(b,LinearEntity) : return self._line_pgon(b,a,**kargs)
    
    if isinstance(a,LinearEntity) and isinstance(b,LinearEntity):
        return self._line_line(a,b)
    
     raise NotImplementedError("I don't know how to intersect these objects.")

In [ ]:
"""
Intersecting Two Segments
The xsec.of() method attempts to intersect any two given objects, and returns a 
Boolean indicating success. If any geometry results from the intersection 
operation, it is accessible via the xsec.results property, or by square-bracket 
index as is in xsec[0].
"""
xsec = Intersector()
if xsec.of(seg_a, seg_b):
    print "segs intersect at ", xsec.results[0]

In [ ]:
"""
Trimming Two Segments
Any useful auxiliary properties that are generated as a by-product of an 
intersection operation are dynamically assigned as members of an Intersector 
object. Here, the auxiliary properties of a Line-Line intersection operation 
are used to rescale the two given Segments in something like a trim operation.
"""
xsec = Intersector()
if xsec.of(seg_a, seg_b):
    print "segs intersect at ", xsec[0]
    seg_a.vec *= xsec.ta
    seg_b.vec *= xsec.tb


## Elemental Intersections

\begin{align}
t^{*} = \frac{-\vec{n} \bullet (\vec{p_{0}} - \vec{q_{0}} ) }{\vec{n} \bullet \vec{v}}
\end{align}

In [ ]:
"""
Line-Line Intersection
Finds intersection of two given LinearEntitites, returning False if the two 
are parallel, or if no intersection is found. Collinear lines are handled 
separately.
"""    
def _line_line(self,ln_a,ln_b):
    # collinear line-line intersections handled separately
    if ln_a.is_collinear(ln_b): return self._line_line_collinear(ln_a,ln_b)
    # parallel lines do not intersect
    if ln_a.is_parallel(ln_b, self.tol): return False
    
    va, vb = ln_a.vec, ln_b.vec
    vqp = Vec(ln_a.spt-ln_b.spt)
    dp = va.dot(vb)
    denom = -va.length2*vb.length2 + dp*dp
    
    self.ta = (vb.length2*(va.dot(vqp)) - dp*(vb.dot(vqp)))/denom
    self.tb = (dp*(va.dot(vqp)) - va.length2*(vb.dot(vqp)))/denom
    pa, pb = ln_a.eval(self.ta), ln_b.eval(self.tb)   
    #if there is an intersection point
    if pa.is_equal(pb, self.tol) :
        #check parameter interval for separate line types
        if type(ln_a) == Ray and self.ta < 0.0 : return False
        if type(ln_b) == Ray and self.tb < 0.0 : return False
        if type(ln_a) == Segment and (self.ta< 0 or self.ta> 1) : return False
        if type(ln_b) == Segment and (self.tb< 0 or self.tb> 1) : return False
        self._geom.append(pa)
        return True
    else: 
        self.log = "No intersection found, recording shortest Segment"
        self._geom.append(Segment(pa,pb))
        return False

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P09.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P10.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Line-Plane Intersection Method
"""
def _line_plane(self,line,plane):
    # Line lies on the Plane
    if plane.contains(line.spt) and plane.contains(line.spt+line.vec):
        # store resulting Line intersection
        self._geom.append(line)
        return True
    
    denom = plane.normal.dot(line.vec) 
    #Line lies in a parallel Plane
    if denom == 0 : return False 
    t = plane.normal.dot(plane.origin-line.spt) / denom
    # {a} dynamically create a new member
    self.dist = t 
    
    #check inclusion of parameter in interval of line type
    if type(line) == Ray and (t < 0.0): return False
    if type(line) == Segment and ((t < 0.0) or t > 1.0): return False
    # store resulting Point intersection
    self._geom.append(line.eval(t))
    return True

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P11.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Plane-Plane Intersection Method
Finds the intersection of two given Planes, returning False if the two are 
parallel and do not intersect.
"""
def _plane_plane(self,pln_a,pln_b):
    # parallel planes do not intersect
    if pln_a.normal.is_parallel(pln_b.normal) : return False
    n1, n2 = pln_a.normal, pln_b.normal
    dot, r1, r2 = n1.dot(n2), n1.dot(pln_a.origin), n2.dot(pln_b.origin)
    vec = n1.cross(pln_b.normal)
    denom = 1-dot*dot
    c1, c2 = (r1 - dot*r2)/denom, (r2 - dot*r1)/denom
    p0 = n1*c1 + n2*c2
    self._geom.append(Line(p0, vec))    
    return True

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P12.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Circle-Plane Intersection Method
Finds the intersection of a Plane and a Circle. First, the intersection of the 
given Plane and the Circle Plane is found. If successful, the problem reduces 
to a Line-Circle intersection problem.
"""
def _circle_plane(self,circ,plane):
    # intersection of the circle plane and the given plane
    xsec = Intersector()
    plane_success = xsec._plane_plane(circ,plane)
    # if planes don't intersect, neither does the circle
    if not plane_success :  return False
    # record the line of plane-plane intersection
    self.line = xsec[0] 
    # nearest point projection
    npt, t, dist = self.line.near(circ.origin)
    # nearest point lies outside circle, no intersection
    if dist > circ.rad: return False   
    # nearest point is on circle, one point of intersection
    if (abs(dist-circ.rad) < self.tol):
        self._geom.append(npt)
        return True  
    # nearest point is in circle, two points of intersection        
    if dist < circ.rad:
        factor = math.sqrt(circ.rad**2-dist**2)/self.line.vec.length
        self._geom.append(npt - self.line.vec*factor)
        self._geom.append(npt + self.line.vec*factor)
        return True

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P13.jpg" style="width: 200px; display: inline;">